In [1]:
import pandas as pd

- **Carga de Datasets**

In [2]:
df_games= pd.read_csv('DataSet_tranformados/games.csv',delimiter = ',',encoding = "utf-8")
df_games.head(10)

,genres,game_name,game_id,release_year
0,Action,Lost Summoner Kitty,761140,2018
1,Casual,Lost Summoner Kitty,761140,2018
2,Indie,Lost Summoner Kitty,761140,2018
3,Simulation,Lost Summoner Kitty,761140,2018
4,Strategy,Lost Summoner Kitty,761140,2018
5,Free to Play,Ironbound,643980,2018
6,Indie,Ironbound,643980,2018
7,RPG,Ironbound,643980,2018
8,Strategy,Ironbound,643980,2018
9,Casual,Real Pool 3D - Poolians,670290,2017


In [3]:
df_reviews= pd.read_csv('DataSet_tranformados/reviews.csv',delimiter = ',',encoding = "utf-8")
df_reviews.head()

,user_id,year,game_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2


In [12]:
df_items= pd.read_parquet('DataSet_tranformados/items.parquet')
df_items.head()

,user_id,game_id,game_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [5]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   game_id           int32 
 2   game_name         object
 3   playtime_forever  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 137.6+ MB


In [75]:
#df= df_games[df_games['genres'].notna()]
#df['genres2']= df['genres'].apply(ast.literal_eval)

In [5]:
def PlayTimeGenre(genero: str):
    df_filtro_genero= df_games[df_games['genres'] == genero]
    df_union = pd.merge(df_filtro_genero, df_items, on='game_name', how= 'inner')
    df_grupo_por_genero_y_año = df_union.groupby(by=['genres','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    year_max_horas= int(df_grupo_por_genero_y_año.iloc[0,1])
    result= [{'Año de lanzamiento con más horas jugadas para Género {}'.format(genero): year_max_horas}]
    return result

In [20]:
PlayTimeGenre('Action')

[{'Año de lanzamiento con más horas jugadas para Género Action': 2012}]

In [50]:
def UserForGenre(genero: str):
    # Se agrupa el dataset "df_Items" por Usuario y juegos y se sumo el tiempo de juef
    df_items_por_user = df_items.groupby(by=['user_id','game_name'], as_index=False)['playtime_forever'].sum() 
    # Se filtra el dataset "df_games" por el genero requerido
    df_filtro_genero= df_games[df_games['genres'] == genero]
    # Se hace una union de los juegos fltrados por el genero requerido y el dataset de items ya filtrado
    df_union = pd.merge(df_filtro_genero, df_items_por_user, on='game_name', how= 'inner')
    # se suman las horas jugadas de cada usuario por año
    df_grupo_por_user_y_año = df_union.groupby(by=['user_id','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    # Se suman las horas jugadas de todos los años por usuario
    df_user_max= df_grupo_por_user_y_año.groupby(by=['user_id'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    user_max_horas= df_user_max.iloc[0,0]   # Se toma el user_id con mas horas jugadas 
    df_horas_jugadas= df_grupo_por_user_y_año[df_grupo_por_user_y_año['user_id']==user_max_horas]   #se realiza un filtro por el usuario con mas horas jugadas
    df_horas_jugadas.sort_values(by='playtime_forever', ascending=False)    #se ordena por tiempo de juego
    
    list_horas_xyear= [{'Año ': year, 'Horas: ': horas} for year, horas in zip(df_horas_jugadas['release_year'], df_horas_jugadas['playtime_forever'])]
    
    result= {'Usuario con mas horas jugadas para el Genero {}'.format(genero): user_max_horas,
            'Horas Jugadas':list_horas_xyear}
    
    return result
    
    

In [51]:
UserForGenre('Action')

{'Usuario con mas horas jugadas para el Genero Action': 'shinomegami',
 'Horas Jugadas': [{'Año ': 2003, 'Horas: ': 532756},
  {'Año ': 2009, 'Horas: ': 209805},
  {'Año ': 2013, 'Horas: ': 148625},
  {'Año ': 2015, 'Horas: ': 102197},
  {'Año ': 2014, 'Horas: ': 92080},
  {'Año ': 2011, 'Horas: ': 89382},
  {'Año ': 2007, 'Horas: ': 68354},
  {'Año ': 2016, 'Horas: ': 42563},
  {'Año ': 2012, 'Horas: ': 18284},
  {'Año ': 2000, 'Horas: ': 10775},
  {'Año ': 2010, 'Horas: ': 9405},
  {'Año ': 1998, 'Horas: ': 3716},
  {'Año ': 2001, 'Horas: ': 2429},
  {'Año ': 2006, 'Horas: ': 2043},
  {'Año ': 1996, 'Horas: ': 1161},
  {'Año ': 2005, 'Horas: ': 343},
  {'Año ': 1999, 'Horas: ': 303},
  {'Año ': 2008, 'Horas: ': 180},
  {'Año ': 1988, 'Horas: ': 136},
  {'Año ': 2017, 'Horas: ': 135},
  {'Año ': 2004, 'Horas: ': 120},
  {'Año ': 1997, 'Horas: ': 99},
  {'Año ': 1995, 'Horas: ': 2},
  {'Año ': 1993, 'Horas: ': 0},
  {'Año ': 1992, 'Horas: ': 0},
  {'Año ': 1991, 'Horas: ': 0},
  {'Año 

In [7]:
def UsersRecommend(año: int):    
    df_filtred_rec_sent_year= df_reviews[(df_reviews['recommend']== True) & (df_reviews['sentiment_analysis']> 0) & (df_reviews['year'] == año)]
    df_grupo_xgames= df_filtred_rec_sent_year.groupby('game_id', as_index=False).size().sort_values(by='size', ascending=False)
    df_games_sin_duplicados = df_games.drop_duplicates(subset=['game_id'])  # se crea una df temporal sin id_games duplicados
    df_union_con_items= pd.merge(df_grupo_xgames, df_games_sin_duplicados, on='game_id', how= 'inner')
    result= {'Puesto {}'.format(pos +1): game for pos, game in zip(range(3),df_union_con_items['game_name'].iloc[:3])}
    return result

In [12]:
UsersRecommend(2010)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Killing Floor',
 'Puesto 3': 'Alien Swarm'}

#########################################################################################################

In [50]:
def Generar_df_funcion1():
    df_union_games_items = pd.merge(df_games, df_items, on='game_name', how= 'inner')
    df_grupo_por_genero_y_año = df_union_games_items.groupby(by=['genres','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    df_grupo_por_genero_y_año.to_csv('DataSet_tranformados/funcion1.csv', index=False)    
    return

In [49]:
Generar_df_funcion1()

In [ ]:
def UserForGenre(genero: str):
    # Se agrupa el dataset "df_Items" por Usuario y juegos y se sumo el tiempo de juef
    df_items_por_user = df_items.groupby(by=['user_id','game_name'], as_index=False)['playtime_forever'].sum() 
    # Se filtra el dataset "df_games" por el genero requerido
    df_filtro_genero= df_games[df_games['genres'] == genero]
    # Se hace una union de los juegos fltrados por el genero requerido y el dataset de items ya filtrado
    df_union = pd.merge(df_filtro_genero, df_items_por_user, on='game_name', how= 'inner')
    # se suman las horas jugadas de cada usuario por año
    df_grupo_por_user_y_año = df_union.groupby(by=['user_id','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    # Se suman las horas jugadas de todos los años por usuario
    df_user_max= df_grupo_por_user_y_año.groupby(by=['user_id'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    user_max_horas= df_user_max.iloc[0,0]   # Se toma el user_id con mas horas jugadas 
    df_horas_jugadas= df_grupo_por_user_y_año[df_grupo_por_user_y_año['user_id']==user_max_horas]   #se realiza un filtro por el usuario con mas horas jugadas
    df_horas_jugadas.sort_values(by='playtime_forever', ascending=False)    #se ordena por tiempo de juego
    
    list_horas_xyear= [{'Año ': year, 'Horas: ': horas} for year, horas in zip(df_horas_jugadas['release_year'], df_horas_jugadas['playtime_forever'])]
    
    result= {'Usuario con mas horas jugadas para el Genero {}'.format(genero): user_max_horas,
            'Horas Jugadas':list_horas_xyear}
    
    return result
    
    

In [13]:
df_items.shape

(5094092, 4)

In [41]:
def Generar_df_funcion2():
    df_union_game_items = pd.merge(df_games, df_items, on='game_id', how= 'inner')
    df_grupo_por_gen_us_year = df_union_game_items.groupby(by=['genres','user_id','release_year'], as_index=False)['playtime_forever'].sum()
    df_grupo_por_gen_us_year.to_parquet('DataSet_tranformados/funcion2.parquet', index=False)
    return

In [42]:
Generar_df_funcion2()

In [80]:
df_union_game_items = pd.merge(df_games, df_items, on='game_id', how= 'inner')

In [56]:
df_union_game_items.shape

(9965664, 7)

In [85]:
df_grupo_por_gen_us_year = df_union_game_items.groupby(by=['genres','user_id','release_year'], as_index=False)['playtime_forever'].sum()
df_grupo_por_gen_us_year.shape

(3495282, 4)

In [18]:
df_union_game_items.head()

,genres,game_name_x,game_id,release_year,user_id,game_name_y,playtime_forever
0,Action,Carmageddon Max Pack,282010,1997,UTNerd24,Carmageddon Max Pack,5
1,Action,Carmageddon Max Pack,282010,1997,I_DID_911_JUST_SAYING,Carmageddon Max Pack,0
2,Action,Carmageddon Max Pack,282010,1997,76561197962104795,Carmageddon Max Pack,0
3,Action,Carmageddon Max Pack,282010,1997,r3ap3r78,Carmageddon Max Pack,0
4,Action,Carmageddon Max Pack,282010,1997,saint556,Carmageddon Max Pack,13


In [61]:
df_grupo_por_user_y_año = df_union_game_items.groupby(by=['genres','user_id','release_year'], as_index=False)['playtime_forever'].sum()

In [62]:
df_grupo_por_user_y_año

,genres,user_id,release_year,playtime_forever
0,Action,--000--,0,4
1,Action,--000--,2009,5329
2,Action,--000--,2010,22
3,Action,--000--,2011,6522
4,Action,--000--,2012,109346
...,...,...,...,...
3495277,Web Publishing,zepavil,2015,9010
3495278,Web Publishing,zeshirky,2007,1
3495279,Web Publishing,zevlupine,2012,4
3495280,Web Publishing,zilaman,2013,9


In [88]:
def UserForGenre2(genero: str):
 
    # Se suman las horas jugadas de todos los años por usuario
    df_filtro_genero= df_grupo_por_gen_us_year[df_grupo_por_gen_us_year['genres'] == genero]
    df_user_max= df_filtro_genero.groupby(by=['user_id','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    
    user_max_horas= df_user_max.iloc[0,0]   # Se toma el user_id con mas horas jugadas 
    df_horas_jugadas= df_user_max[df_user_max['user_id']==user_max_horas]   #se realiza un filtro por el usuario con mas horas jugadas
    df_horas_jugadas.sort_values(by='playtime_forever', ascending=False)    #se ordena por tiempo de juego
    
    list_horas_xyear= [{'Año ': year, 'Horas: ': horas} for year, horas in zip(df_horas_jugadas['release_year'], df_horas_jugadas['playtime_forever'])]
    
    result= {'Usuario con mas horas jugadas para el Genero {}'.format(genero): user_max_horas,
            'Horas Jugadas':list_horas_xyear}
    return result

In [83]:
    # Se suman las horas jugadas de todos los años por usuario
df_filtro_genero= df_grupo_por_user_y_año[df_grupo_por_user_y_año['genres'] == 'Action']
df_user_max= df_filtro_genero.groupby(by=['user_id','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
df_user_max

,user_id,release_year,playtime_forever
415109,Evilutional,2012,680991
661750,shinomegami,2003,532756
23285,76561197977470391,2004,529351
680074,theoriginalmasta,2004,479261
75389,76561198027114566,2004,411858
...,...,...,...
619532,mattswtmn,2016,0
619530,mattswtmn,2014,0
296471,76561198081247765,2016,0
296489,76561198081250074,2017,0


In [84]:
user_max_horas= df_user_max.iloc[0,0]   # Se toma el user_id con mas horas jugadas 
df_horas_jugadas= df_user_max[df_user_max['user_id']==user_max_horas]   #se realiza un filtro por el usuario con mas horas jugadas
df_horas_jugadas.sort_values(by='playtime_forever', ascending=False)    #se ordena por tiempo de juego
df_horas_jugadas

,user_id,release_year,playtime_forever
415109,Evilutional,2012,680991
415110,Evilutional,2013,69726
415111,Evilutional,2014,24226
415114,Evilutional,2017,10894
415107,Evilutional,2010,4102
415106,Evilutional,2009,2037
415108,Evilutional,2011,1968
415113,Evilutional,2016,1291
415112,Evilutional,2015,112
415104,Evilutional,2003,0


In [92]:
UserForGenre('Strategy')

{'Usuario con mas horas jugadas para el Genero Strategy': 'shinomegami',
 'Horas Jugadas': [{'Año ': 2003, 'Horas: ': 531614},
  {'Año ': 2013, 'Horas: ': 139630},
  {'Año ': 2014, 'Horas: ': 124901},
  {'Año ': 2012, 'Horas: ': 79098},
  {'Año ': 2007, 'Horas: ': 68627},
  {'Año ': 1993, 'Horas: ': 22001},
  {'Año ': 0, 'Horas: ': 18783},
  {'Año ': 2010, 'Horas: ': 13733},
  {'Año ': 2009, 'Horas: ': 8434},
  {'Año ': 2011, 'Horas: ': 6882},
  {'Año ': 2015, 'Horas: ': 3270},
  {'Año ': 2002, 'Horas: ': 2434},
  {'Año ': 2016, 'Horas: ': 1379},
  {'Año ': 2008, 'Horas: ': 356},
  {'Año ': 2004, 'Horas: ': 76},
  {'Año ': 1999, 'Horas: ': 14},
  {'Año ': 2001, 'Horas: ': 2},
  {'Año ': 1997, 'Horas: ': 0},
  {'Año ': 1998, 'Horas: ': 0},
  {'Año ': 2006, 'Horas: ': 0},
  {'Año ': 2005, 'Horas: ': 0},
  {'Año ': 2017, 'Horas: ': 0}]}

In [91]:
UserForGenre2('Strategy')

{'Usuario con mas horas jugadas para el Genero Strategy': 'shinomegami',
 'Horas Jugadas': [{'Año ': 2003, 'Horas: ': 531616},
  {'Año ': 2013, 'Horas: ': 147508},
  {'Año ': 2014, 'Horas: ': 127735},
  {'Año ': 2012, 'Horas: ': 82587},
  {'Año ': 2009, 'Horas: ': 74762},
  {'Año ': 2007, 'Horas: ': 68861},
  {'Año ': 0, 'Horas: ': 37096},
  {'Año ': 1993, 'Horas: ': 22001},
  {'Año ': 1995, 'Horas: ': 20485},
  {'Año ': 2010, 'Horas: ': 14092},
  {'Año ': 2011, 'Horas: ': 6385},
  {'Año ': 2015, 'Horas: ': 3407},
  {'Año ': 2002, 'Horas: ': 2434},
  {'Año ': 2016, 'Horas: ': 1379},
  {'Año ': 2004, 'Horas: ': 824},
  {'Año ': 2008, 'Horas: ': 356},
  {'Año ': 1999, 'Horas: ': 14},
  {'Año ': 2001, 'Horas: ': 2},
  {'Año ': 2006, 'Horas: ': 2},
  {'Año ': 2017, 'Horas: ': 0},
  {'Año ': 2005, 'Horas: ': 0},
  {'Año ': 2000, 'Horas: ': 0},
  {'Año ': 1998, 'Horas: ': 0},
  {'Año ': 1997, 'Horas: ': 0}]}